In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import Version, FuncGroups, CollisionType, VERSION_DEFAULT
from utils import OUTPUT_PATH


def count_dup(v: Version):
    functions = FuncGroups.from_dump(v.dwarf_funcs_path)
    results = {c: FuncGroups() for c in CollisionType}

    for name, group in functions.iter_groups():
        results[group.get_collision_type()].add_group(name, group)

    path = OUTPUT_PATH / "dup" / v.name
    for c in CollisionType:
        results[c].save_groups(path / f"{c.name.lower()}.txt")

    return {k: v.num_groups for k, v in results.items()}


# count_dup(VERSION_DEFAULT)

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.


In [3]:
from depsurf import VersionGroup
from utils import save_pkl

data = {}

for v in VersionGroup.LTS:
    data[v] = count_dup(v)

save_pkl(data, "dup")

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl
[       groups.py:56 ] INFO: Saved FuncGroups(17237 groups, 17237 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/unique_global.txt
[       groups.py:56 ] INFO: Saved FuncGroups(35710 groups, 35710 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/unique_static.txt
[       groups.py:56 ] INFO: Saved FuncGroups(4042 groups, 53294 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/include_dup.txt
[       groups.py:56 ] INFO: Saved FuncGroups(404 groups, 961 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/static_static.txt
[       groups.py:56 ] INFO: Saved FuncGroups(10 groups, 20 functions) to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/static_global.txt
[       groups.py:68 ] INFO: Loading funcs from /Users/

In [9]:
from utils import load_pkl, save_latex
import pandas as pd


data = load_pkl("dup")

result = {}
for k, v in data.items():
    result[("Linux Kernel Version", r"\multicolumn{1}{c}{" + k.short_version + "}")] = v

df = pd.DataFrame(result)
df = df.map(lambda v: f"{v / 1000:.1f}k" if v > 1000 else v)
latex = df.to_latex(multicolumn_format="c", column_format="l|rrrrr")
save_latex(latex, "dup")
df

[          pkl.py:18 ] INFO: Loding dup from /Users/szhong/Downloads/bpf-study/output/dup.pkl
[        latex.py:19 ] INFO: Saved dup to /Users/szhong/Downloads/bpf-study/paper/tabs/dup.tex


Linux Kernel Version                                                                                                  
                          \multicolumn{1}{c}{4.4} \multicolumn{1}{c}{4.15} \multicolumn{1}{c}{5.4} \multicolumn{1}{c}{5.15} \multicolumn{1}{c}{6.8}
Unique Global                               17.2k                    20.1k                   22.7k                    26.6k                   31.5k
Unique Static                               35.7k                    41.7k                   48.2k                    53.3k                   60.2k
Static Dup. with #include                    4.0k                     4.8k                    5.5k                     6.2k                    7.4k
Static-Static Collision                       404                      398                     411                      444                     498
Static-Global Collision                        10                       26                      27                       26                      29